# Nabewerking klimaatsommen prep
De klimaatsommen hebben rasters nodig.

Deze worden uit de 18 gedownloade netcdf bestanden gemaakt. Hiervoor is het nodig 
- dat er lokaal een wateschadeschatter is geinstalleerd.
- panden beschikbaar in 01_source_data\panden.gpkg. Maak een clip van "E:\01.basisgegevens\Panden\Panden_HHNK_sampled_v2.gpkg" met de poldershape

In [ ]:
import os
import json
try:
    with open(os.getcwd() + "/notebook_data.json") as f:
        notebook_data = json.load(f)
except:
    pass

import geopandas as gpd

from hhnk_threedi_tools import Folders
import hhnk_research_tools as hrt
from hhnk_research_tools.raster_functions import reproject

from hhnk_threedi_tools.core.result_rasters.netcdf_to_gridgpkg import ThreediGrid
from hhnk_threedi_tools.core.result_rasters.calculate_raster import BaseCalculatorGPKG
import ipywidgets as widgets


# Folders inladen
folder = Folders(notebook_data['polder_folder'])

# Of handmatig;
# folder=Folders(r"E:\02.modellen\model_test_v2")


In [ ]:
def item_layout(width="95%", grid_area="", **kwargs):
    return widgets.Layout(
        width=width, grid_area=grid_area, **kwargs
    )  # override the default width of the button to 'auto' to let the button grow

dem_path = folder.model.schema_base.rasters.dem.path
dem = hrt.Raster(dem_path)

output_folder_options = [""] + folder.threedi_results.batch.content
output_folder_box = widgets.Select(
    options=output_folder_options,
    rows=len(output_folder_options),
    disabled=False,
    layout=item_layout(grid_area="output_folder_box"),
)

print("Selecteer map met batch resultaten")
display(output_folder_box)


## Dem res 0.5m afdwingen

In [ ]:
#When resolution is not 0.5 we reproject DEM so we do all calculations on 0.5m.
if output_folder_box.value == "":
    raise Exception("Select batch folder")

batch_fd = folder.threedi_results.batch[output_folder_box.value]
        
if dem.metadata.pixel_width != 0.5:
    new_dem_path = batch_fd.downloads.pl/f"{dem.pl.stem}_05m.tif"
    if not new_dem_path.exists():
        reproject(src = dem, 
                    target_res = 0.5,
                    output_path = new_dem_path
        )
if dem.metadata.pixel_width != 0.5:
    new_dem_path = batch_fd.downloads.pl/f"{dem.pl.stem}_05m.tif"
    dem = hrt.Raster(new_dem_path)

## Klaarzetten grid

In [ ]:
for name in batch_fd.downloads.names:
    print(name)
    scenario = getattr(batch_fd.downloads, name)

    threedi_result = scenario.netcdf

    #Select result
    threedigrid = ThreediGrid(folder=folder, threedi_result=threedi_result)

    #Convert netcdf to grid gpkg
    threedigrid.netcdf_to_grid_gpkg()

    #Replace waterlevel of selected cells with avg of neighbours.
    threedigrid.waterlevel_correction(output_col="wlvl_max_replaced")

# Diepterasters

In [ ]:
OVERWRITE=False

for name in batch_fd.downloads.names:
    print(name)
    
    scenario = getattr(batch_fd.downloads, name)
    
    threedi_result = scenario.netcdf


    grid_gdf = gpd.read_file(threedi_result.pl/"grid_corr.gpkg", driver="GPKG")


    calculator_kwargs = {"dem_path":dem.source_path,
                            "grid_gdf":grid_gdf, 
                            "wlvl_column":"wlvl_max_replaced"}

    #Init calculator
    with BaseCalculatorGPKG(**calculator_kwargs) as self:
        self.run(output_folder=scenario.depth_max.pl.parent,  
                    output_raster_name=scenario.depth_max.pl.name,
                    mode="MODE_WDEPTH",
                    overwrite=OVERWRITE)
        print("Done.")

# Schaderasters

In [ ]:
# Schadeschatter heeft wat extra voorbereiding nodig.
from pathlib import Path
schadeschatter_path = Path(r"E:\01.basisgegevens\hhnk_schadeschatter")

import sys
if str(schadeschatter_path) not in sys.path:
    sys.path.append(str(schadeschatter_path))
import hhnk_schadeschatter as hhnk_wss


for name in batch_fd.downloads.names:
    print(name)
    scenario = getattr(batch_fd.downloads, name)
    
    #Variables
    # cfg_file = schadeschatter_path/'01_data/cfg/cfg_hhnk_2020.cfg'
    cfg_file = schadeschatter_path/'01_data/cfg/cfg_lizard.cfg'
    landuse_file = r"E:\01.basisgegevens\rasters\landgebruik/landuse2020_tiles/combined_rasters.vrt"

    depth_file = scenario.depth_max.path
    output_raster = scenario.damage_total

    wss_settings = {'inundation_period': 48, #uren
                    'herstelperiode':'10 dagen',
                    'maand':'sep',
                    'cfg_file':cfg_file,
                    'dmg_type':'gem'}

    #Calculation
    self = hhnk_wss.wss_main.Waterschadeschatter(depth_file=depth_file, 
                            landuse_file=landuse_file, 
                            wss_settings=wss_settings)

    # Berekenen schaderaster
    self.run(output_raster=output_raster, 
            calculation_type="sum", 
            overwrite=False)


# Waterstandrasters (optioneel)
Nog niet nodig voor standaard nabewerking

In [ ]:
OVERWRITE=False


for name in batch_fd.downloads.names:
    print(name)
    
    scenario = getattr(batch_fd.downloads, name)
    
    threedi_result = scenario.netcdf


    grid_gdf = gpd.read_file(threedi_result.pl/"grid_corr.gpkg", driver="GPKG")


    calculator_kwargs = {"dem_path":dem.source_path,
                            "grid_gdf":grid_gdf, 
                            "wlvl_column":"wlvl_max_replaced"}

    #Init calculator
    with BaseCalculatorGPKG(**calculator_kwargs) as self:
        self.run(output_folder=scenario.wlvl_max.pl.parent,  
                    output_raster_name=scenario.wlvl_max.pl.name,
                    mode="MODE_WLVL",
                    overwrite=OVERWRITE)
        print("Done.")
